# Building a Lead Scoring Model with EvalML

In this demo, we will build an optimized lead scoring model using EvalML. To optimize the pipeline, we will set up an objective function to maximize the revenue generated with true positives while taking into account the cost of false positives. At the end of this demo, we also show you how introducing the right objective during the training is over 6x better than using a generic machine learning metric like AUC.

In [1]:
import evalml
from evalml.objectives import LeadScoring

## Configure LeadScoring 

To optimize the pipelines toward the specific business needs of this model, you can set your own assumptions for how much value is gained through true positives and the cost associated with false positives. These parameters are

* `true_positive` - dollar amount to be gained with a successful lead
* `false_positive` - dollar amount to be lost with an unsuccessful lead

Using these parameters, EvalML builds a pileline that will maximize the amount of revenue per lead generated.

In [2]:
lead_scoring_objective = LeadScoring(
    true_positives=1000,
    false_positives=-10
)

## Dataset

We will be utilizing a dataset detailing a customer's job, country, state, zip, online action, the dollar amount of that action and whether they were a successful lead.

In [3]:
import pandas as pd

customers = pd.read_csv('s3://featurelabs-static/lead_scoring_ml_apps/customers.csv')
interactions = pd.read_csv('s3://featurelabs-static/lead_scoring_ml_apps/interactions.csv')
leads = pd.read_csv('s3://featurelabs-static/lead_scoring_ml_apps/previous_leads.csv')

X = customers.merge(interactions, on='customer_id').merge(leads, on='customer_id')
y = X['label']

X = X.drop(['customer_id', 'date_registered', 'birthday','phone', 'email',
        'owner', 'company', 'id', 'time_x',
        'session', 'referrer', 'time_y', 'label'], axis=1)

display(X.head())

,job,country,state,zip,action,amount
0,"Engineer, mining",NaN,NY,60091.0,page_view,NaN
1,"Psychologist, forensic",US,CA,NaN,purchase,135.23
2,"Psychologist, forensic",US,CA,NaN,page_view,NaN
3,Air cabin crew,US,NaN,60091.0,download,NaN
4,Air cabin crew,US,NaN,60091.0,page_view,NaN


## Search for best pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

EvalML natively supports one-hot encoding and imputation so the above `NaN` and categorical values will be taken care of.

In [4]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=0.2, random_state=0)

print(X.dtypes)

job         object
country     object
state       object
zip        float64
action      object
amount     float64
dtype: object


Because the lead scoring labels are binary, we will use `AutoClassifier`. When we call `.fit()`, the search for the best pipeline will begin. 

In [5]:
clf = evalml.AutoClassifier(objective=lead_scoring_objective,
                            additional_objectives=['auc'],
                            max_pipelines=5)

clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Lead Scoring. Greater score is better.

Searching up to 5 pipelines. 
Possible model types: xgboost, random_forest, linear_model

✔ XGBoost Classifier w/ One Hot Encod...     0%|          | Elapsed:00:05
✔ XGBoost Classifier w/ One Hot Encod...    20%|██        | Elapsed:00:10
✔ Random Forest Classifier w/ One Hot...    40%|████      | Elapsed:00:21
✔ XGBoost Classifier w/ One Hot Encod...    60%|██████    | Elapsed:00:26
✔ Logistic Regression Classifier w/ O...    80%|████████  | Elapsed:00:32
✔ Logistic Regression Classifier w/ O...   100%|██████████| Elapsed:00:32

✔ Optimization finished


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the lead scoring objective we defined

In [6]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,4,LogisticRegressionPipeline,42.988250,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."
1,0,XGBoostPipeline,42.140250,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
2,3,XGBoostPipeline,42.140250,False,"{'eta': 0.5288949197529046, 'min_child_weight'..."
3,1,XGBoostPipeline,42.101515,False,"{'eta': 0.38438170729269994, 'min_child_weight..."
4,2,RFClassificationPipeline,41.146060,False,"{'n_estimators': 569, 'max_depth': 22, 'impute..."


to select the best pipeline we can run

In [7]:
best_pipeline = clf.best_pipeline

### Describe pipeline

You can get more details about any pipeline. Including how it performed on other objective functions.

In [8]:
clf.describe_pipeline(clf.rankings.iloc[0]["id"])

****************************************************************************************
* Logistic Regression Classifier w/ One Hot Encoder + Simple Imputer + Standard Scaler *
****************************************************************************************

Problem Types: Binary Classification, Multiclass Classification
Model Type: Linear Model
Objective to Optimize: Lead Scoring (greater is better)
Number of features: 229

Pipeline Steps
1. One Hot Encoder
2. Simple Imputer
	 * impute_strategy : most_frequent
3. Standard Scaler
4. Logistic Regression Classifier
	 * penalty : l2
	 * C : 8.444214828324364

Training
Training for Binary Classification problems.
Total training time (including CV): 6.9 seconds

Cross Validation
----------------
             Lead Scoring   AUC # Training # Testing
0                  39.652 0.909   3099.000  1550.000
1                  43.948 0.908   3099.000  1550.000
2                  45.365 0.926   3100.000  1549.000
mean               42.988 0.

## Evaluate on hold out

Finally, we retrain the best pipeline on all of the training data and evaluate on the holdout

In [9]:
best_pipeline.fit(X_train, y_train)

Now, we can score the pipeline on the hold out data using both the lead scoring score and the AUC.

In [10]:
best_pipeline.score(X_holdout, y_holdout, other_objectives=["auc", lead_scoring_objective])

(45.55460017196904,
 OrderedDict([('AUC', 0.8962224236929586),
              ('Lead Scoring', 45.55460017196904)]))

## Why optimize for a problem-specific objective?

To demonstrate the importance of optimizing for the right objective, let's search for another pipeline using AUC, a common machine learning metric. After that, we will score the holdout data using the lead scoring objective to see how the best pipelines compare.

In [11]:
clf_auc = evalml.AutoClassifier(objective='auc',
                                additional_objectives=[],
                                max_pipelines=5)

clf_auc.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. Greater score is better.

Searching up to 5 pipelines. 
Possible model types: xgboost, random_forest, linear_model

✔ XGBoost Classifier w/ One Hot Encod...     0%|          | Elapsed:00:02
✔ XGBoost Classifier w/ One Hot Encod...    20%|██        | Elapsed:00:05
✔ Random Forest Classifier w/ One Hot...    40%|████      | Elapsed:00:12
✔ XGBoost Classifier w/ One Hot Encod...    60%|██████    | Elapsed:00:15
✔ Logistic Regression Classifier w/ O...    80%|████████  | Elapsed:00:21
✔ Logistic Regression Classifier w/ O...   100%|██████████| Elapsed:00:21

✔ Optimization finished


like before, we can look at the rankings and pick the best pipeline

In [12]:
clf_auc.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,4,LogisticRegressionPipeline,0.920612,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."
1,2,RFClassificationPipeline,0.548868,False,"{'n_estimators': 569, 'max_depth': 22, 'impute..."
2,1,XGBoostPipeline,0.545493,False,"{'eta': 0.38438170729269994, 'min_child_weight..."
3,0,XGBoostPipeline,0.515050,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
4,3,XGBoostPipeline,0.500000,False,"{'eta': 0.5288949197529046, 'min_child_weight'..."


In [13]:
best_pipeline_auc = clf_auc.best_pipeline

# train on the full training data
best_pipeline_auc.fit(X_train, y_train)

In [14]:
# get the fraud score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout,  other_objectives=["auc", lead_scoring_objective])

(0.9272136597159263,
 OrderedDict([('AUC', 0.9272136597159263),
              ('Lead Scoring', 7.67841788478074)]))

When we optimize for AUC, we can see that the AUC score from this pipeline is better than the AUC score from the pipeline optimized for lead scoring. However, the revenue per lead gained was only `$7` per lead when optimized for AUC and was `$45` when optimized for lead scoring. As a result, we would gain up to 6x the amount of revenue if we optimized for lead scoring.

This happens because optimizing for AUC does not take into account the user-specified true_positive (dollar amount to be gained with a successful lead) and false_positive (dollar amount to be lost with an unsuccessful lead) values. Thus, the best pipelines may produce the highest AUC but may not actually generate the most revenue through lead scoring.

This example highlights how performance in the real world can diverge greatly from machine learning metrics.